# Paper QA Emulation Tool   

> We here emulate the workflow of the PaperQA system by Andrew White (https://thewhitelab.org/). This is a Retrieval Augmented Generation (RAG) application using a Map-Reduce model where we query our embedded index for based on a question, we then write summaries for each returned document based on relevance to the underlying question. Finally, we synthesize each summary into an essay presented as an answer to the original question. 

In [7]:
#| default_exp tools.paperqa_emulation_tool

In [2]:
#| hide
from nbdev import *

In [ ]:
#| export

from alhazen.core import OllamaRunner, PromptTemplateRegistry, get_langchain_llm, get_cached_gguf, \
    get_langchain_embeddings, GGUF_LOOKUP_URL, MODEL_TYPE
from alhazen.tools.basic import AlhazenToolMixin
from alhazen.utils.output_parsers import JsonEnclosedByTextOutputParser
from alhazen.utils.ceifns_db import *
from alhazen.schema_sqla import ScientificKnowledgeCollection, \
    ScientificKnowledgeExpression, ScientificKnowledgeCollectionHasMembers, \
    ScientificKnowledgeItem, ScientificKnowledgeExpressionHasRepresentation, \
    ScientificKnowledgeFragment, ScientificKnowledgeItemHasPart, \
    InformationResource, Note, \
    ScientificKnowledgeCollectionHasNotes, ScientificKnowledgeExpressionHasNotes, \
    ScientificKnowledgeItemHasNotes, ScientificKnowledgeFragmentHasNotes 
from datetime import datetime
from importlib_resources import files
import json

from langchain.callbacks.tracers import ConsoleCallbackHandler
from langchain.chat_models import ChatOllama
from langchain.embeddings import HuggingFaceBgeEmbeddings
from langchain.memory import ConversationBufferMemory
from langchain.prompts import PromptTemplate, ChatPromptTemplate
from langchain.pydantic_v1 import BaseModel, Field, root_validator
from langchain.schema import get_buffer_string, StrOutputParser, OutputParserException, format_document
from langchain.schema.runnable import RunnableParallel, RunnablePassthrough, RunnableLambda
from langchain.tools import BaseTool, StructuredTool
from langchain.vectorstores.pgvector import PGVector
from langchain_core.output_parsers import StrOutputParser, JsonOutputParser

import local_resources.prompt_elements as prompt_elements
import local_resources.linkml as linkml
from operator import itemgetter
import os
import re
import regex
from sqlalchemy import create_engine, exists
from sqlalchemy.orm import sessionmaker, aliased
from time import time,sleep
from typing import Optional, Type
from urllib.request import urlopen
from urllib.parse import quote_plus, quote, unquote
import uuid
import yaml

In [4]:
#| export 

skc = aliased(ScientificKnowledgeCollection)
skc_hm = aliased(ScientificKnowledgeCollectionHasMembers)
ske = aliased(ScientificKnowledgeExpression)
ske_hr = aliased(ScientificKnowledgeExpressionHasRepresentation)
ski = aliased(ScientificKnowledgeItem)
ski_hp = aliased(ScientificKnowledgeItemHasPart)
skf = aliased(ScientificKnowledgeFragment)
n = aliased(Note)
skc_hn = aliased(ScientificKnowledgeCollectionHasNotes)
ske_hn = aliased(ScientificKnowledgeExpressionHasNotes)
ski_hn = aliased(ScientificKnowledgeItemHasNotes)
skf_hn = aliased(ScientificKnowledgeFragmentHasNotes)

In [ ]:
#| export

class PaperQAEmulationToolSchema(BaseModel):
    question: str = Field(description="The question to be considered by this workflow.")
    n_sample_size: Optional[int] = Field(description="The number of documents queried from the index to be evaluated for relevance.")
    n_summary_size: Optional[int] = Field(description="The number of documents queried from the index to be used in generating an answer.")
    collection_id: Optional[int] = Field(None, description="The identifier of the collection to be used to answer the question.")

class PaperQAEmulationTool(BaseTool, AlhazenToolMixin):
    '''Runs a Map-Reduce model where we write a short essay to answer a scientific question based on a set of supporting documents.'''
    name = 'simple_qa_over_papers'
    description = '''Runs a Map-Reduce model where we write a short essay to answer a scientific question based on a set of supporting documents.'''
    args_schema = PaperQAEmulationToolSchema
    
    def _run(self, question, n_sample_size=15, n_summary_size=5, collection_id=-1):
        '''Runs the metadata extraction pipeline over a specified paper.'''
        
        DEFAULT_DOCUMENT_PROMPT = PromptTemplate.from_template(template='"DOI": "{e_id}","CITATION": "{citation}", "CONTENT":"{page_content}"')
        def _combine_documents(docs):
            m = [{'CONTENT':d.page_content, 'DOI':d.metadata.get('e_id'), "CITATION": d.metadata.get('citation')} for d in docs]
            return json.dumps(m)

        #~~~~~~~~~~~~~~~~~~~~~~
        # 1. Set up environment
        #~~~~~~~~~~~~~~~~~~~~~~
        if os.environ.get('LOCAL_FILE_PATH') is None: 
            raise Exception('Where are you storing your local literature database?')
        loc = os.environ['LOCAL_FILE_PATH']
        if loc[-1:] != '/':
            loc += '/'
        if os.environ.get('ALHAZEN_DB_NAME') is None: 
            raise Exception('Which database do you want to use for this application?')
        db_name = os.environ['ALHAZEN_DB_NAME']
        os.environ['PGVECTOR_CONNECTION_STRING'] = "postgresql+psycopg2:///"+db_name
            
        vectorstore = PGVector.from_existing_index(
                embedding = self.db.embed_model, 
                collection_name = 'ScienceKnowledgeItem') 
        
        # set default values for optional parameters
        retriever = vectorstore.as_retriever(search_kwargs={'k':n_sample_size})
        if collection_id != -1:
            retriever = vectorstore.as_retriever(search_kwargs={'k':n_sample_size, 'filter': {'c_ids': 2}})

        #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
        # 2. Run Map Function - Iterate over 'pages' of Documents returned from vectorstore to generate 
        #                       summaries
        #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
                                
        pts = PromptTemplateRegistry()
        pts.load_prompts_from_yaml('paperqa_emulation.yaml')
        pt = pts.get_prompt_template('summarize paper set').generate_chat_prompt_template()
        
        context_build_chain = (
            RunnableParallel({
                "k": itemgetter("k"),
                "question": itemgetter("question"),
                "context": itemgetter("question") | retriever | _combine_documents 
            }
        ))

        summary_chain = (
            RunnableParallel({
                "k": itemgetter("k"),
                "question": itemgetter("question"),
                "summary_length": itemgetter("summary_length"),
                "context": itemgetter("context"),
            })
            | {
                "summary": pt | ChatOllama(model='mixtral') | JsonEnclosedByTextOutputParser(),
                "context": itemgetter("context"),
            }
        )

        input = {'question': question, 'summary_length': 1000, 'k':5}    
        out = context_build_chain.invoke(input, config={'callbacks': [ConsoleCallbackHandler()]})
        context = json.loads(out.get('context'))
        print(len(context))

        page_count = int(n_sample_size / n_summary_size)
        summaries = {}
        for pg in range(page_count):
            paged_context = [context[i] for i in range(pg*n_summary_size, (pg+1)*n_summary_size)]
            paged_input = input.copy()
            paged_input['context'] = json.dumps(paged_context)
            out2 = summary_chain.invoke(paged_input, config={'callbacks': [ConsoleCallbackHandler()]})
            summs = out2.get('summary', [])
            if summs:
                for m in out2.get('summary', []):
                    summaries[m.get('DOI')] = {}
                    for k in m:
                        summaries[m.get('DOI')][k] = m[k]
                for m2 in json.loads(out2.get('context','[]')):
                    if summaries.get(m2.get('DOI')):
                        summaries[m2.get('DOI')]['CITATION'] = m2.get('CITATION')
                        summaries[m2.get('DOI')]['CONTENT'] = m2.get('CONTENT')

        #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
        # 3. Order sumaries in terms of 'RELEVANCE'
        #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
        ordered_summaries = []
        for i, s in enumerate(sorted(summaries.values(), key=lambda x: int(x['RELEVANCE SCORE']), reverse=True)):
            if int(s['RELEVANCE SCORE']) < 7:
                continue
            s['ID'] = i+1
            ordered_summaries.append({'ID':s.get('ID'),
                                        'CITATION': s.get('CITATION'), 
                                        'SUMMARY': s.get('SUMMARY'), 
                                        'RELEVANCE': s.get('RELEVANCE SCORE')})
            
        #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
        # 4. Run Reduce Function - Write the synthesis over summaries provided
        #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
        pt2 = pts.get_prompt_template('write synthesis').generate_chat_prompt_template()

        qa_synthesis_chain = (
            RunnableParallel({
                "question": itemgetter("question"),
                "context": itemgetter("context"),
            })
            | {
                "answer": pt2 | ChatOllama(model='mixtral'),
                "context": itemgetter("context"),
            }
        )

        input = {'question': question, 'context': ordered_summaries[:5]}    
        out3 = qa_synthesis_chain.invoke(input, config={'callbacks': [ConsoleCallbackHandler()]})

        answer = {'answer':out3.get('answer').content,
                'context':ordered_summaries}

        return answer